In [1]:
import os
import pandas as pd


pos_path = ".\\data\\train\\pos\\"
neg_path = ".\\data\\train\\neg\\"

neg_review_list = []
pos_review_list = []
review_list = []
test_set = []

for file in os.listdir(neg_path):
    file_path = os.path.join(neg_path, file)
    fh = open(file_path, 'r', encoding="utf8")
    entry = {'class': 0, 'raw_txt': fh.read()}
    neg_review_list.append(entry)
    fh.close()
    
for file in os.listdir(pos_path):
    file_path = os.path.join(pos_path, file)
    fh = open(file_path, 'r', encoding="utf8")
    entry = {'class': 1, 'raw_txt': fh.read()}
    pos_review_list.append(entry)
    fh.close()

review_list = pd.DataFrame(neg_review_list + pos_review_list)

print("Done")

Done


In [2]:
sample_review = pd.DataFrame(neg_review_list[:10]+pos_review_list[:10])

In [37]:
import re
from nltk.corpus import stopwords
import numpy as np
from textblob import Word, TextBlob

# stopWords = set(stopwords.words('english'))
stopWords=['in','of','at','a','the']

def text_processing(txt):
    processed = re.sub(r'[^\w\s]', '', txt.lower())
    #processed = ' '.join([Word(word).lemmatize() for word in processed.split()])
    processed = negationHandling(processed.split())
    return processed


def negationHandling(word_list):
    """
    given an matrix of individual words in order, combine "negation,"word" into
    "neg_word"
    """
    negation_words = ("not", "hardly", "barely", "never", "neither", "scarcely",
                      "doesn't", "doesnt", "isn't", "isnt", "wasn't", "wasnt",
                      "shouldn't", "shouldnt", "wouldn't", "wouldnt", "couldn't",
                      "couldnt", "won't", "wont", "can't", "cant", "don't", "dont")
    for index in range(len(word_list) - 1):
        if word_list[index] in negation_words:
            word_list[index + 1] = "neg_" + word_list[index + 1]
    new_list = []
    for word in word_list:
        if word not in negation_words:
            new_list.append(word)
    return ' '.join(new_list)


def processing(df):
    # lowering, removing punctuation and lemmatization
    df['processed'] = df['raw_txt'].apply(lambda x: text_processing(x))

    # Removing stopwords    
    df['processed_no_stopwords'] = df['processed'].apply(
        lambda x: ' '.join([t for t in x.split(' ') if t not in stopWords]))
    # Sentiment
    df['sentiment'] = df['processed_no_stopwords'].apply(lambda x: TextBlob(x).sentiment[0])

    return df

sample = processing(sample_review)
df = processing(review_list)
print(sample.head())


   class                                            raw_txt  \
0      0  Story of a man who has unnatural feelings for ...   
1      0  Airport '77 starts as a brand new luxury 747 p...   
2      0  This film lacked something I couldn't put my f...   
3      0  Sorry everyone,,, I know this is supposed to b...   
4      0  When I was little my parents took me along to ...   

                                           processed  \
0  story of a man who has unnatural feelings for ...   
1  airport 77 starts as a brand new luxury 747 pl...   
2  this film lacked something i neg_put my finger...   
3  sorry everyone i know this is supposed to be a...   
4  when i was little my parents took me along to ...   

                              processed_no_stopwords  sentiment  
0  story man who has unnatural feelings for pig s...  -0.071759  
1  airport 77 starts as brand new luxury 747 plan...   0.036677  
2  this film lacked something i neg_put my finger...   0.079167  
3  sorry everyone i 

In [38]:
from sklearn.model_selection import train_test_split

features = [f for f in df.columns.values if f not in ['class']]
x_train, x_test, y_train, y_test = train_test_split(df[features], df['class'],
                                                    test_size=0.20, random_state=10)
x_set = df[features]
target_set = df['class']
print(x_train.head())

                                                 raw_txt  \
5610   Devil Hunter gained notoriety for the fact tha...   
3742   What can I say about Seven Pounds...well I wat...   
5692   This film was so predictable, that during the ...   
22213  In an attempt to cash in on the success of Uni...   
23165  Dark comedy? Gallows humor? How does one make ...   

                                               processed  \
5610   devil hunter gained notoriety for the fact tha...   
3742   what can i say about seven poundswell i watche...   
5692   this film was so predictable that during the e...   
22213  in an attempt to cash in on the success of uni...   
23165  dark comedy gallows humor how does one make a ...   

                                  processed_no_stopwords  sentiment  
5610   devil hunter gained notoriety for fact that it...   0.044228  
3742   what can i say about seven poundswell i watche...  -0.148864  
5692   this film was so predictable that during entir...   0.028685 

In [5]:

from sklearn.base import BaseEstimator, TransformerMixin

class TxtPicker(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
tf_id_feature = Pipeline([
    ('selector', TxtPicker(key='processed_no_stopwords')),
    ('tfidf', CountVectorizer(ngram_range=(1, 3), binary=True)),
    ('top_word_selection', SelectFromModel(LinearSVC()))
    # ('selection', SelectPercentile(score_func=chi2, percentile=50))
])

binary_count_feature = Pipeline([
    ('selector', TxtPicker(key='processed')),            
    ('count', CountVectorizer(ngram_range=(1, 3),binary=True))
])





In [43]:
from sklearn import tree
from sklearn.feature_selection import chi2, SelectPercentile
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC

svc_pipeline = Pipeline([
    ('features', tf_id_feature),
    ('classifier', LinearSVC(C = 0.01))])

logistic_pipeline = Pipeline([
    ('features', tf_id_feature),
    ('classifier', LogisticRegression(C=10))])

decision_tree_pipeline = Pipeline([
    ('feats', feats_tfid),
    ('classifier', tree.DecisionTreeClassifier())])

def fit_predict(model):
    model.fit(x_train, y_train)
    preds = model.predict(x_test)
    print(np.mean(preds == y_test))
    
fit_predict(logistic_pipeline)


C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8828


In [41]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

Cs = [0.001, 0.01, 0.1, 1, 10, 20]
param_grid = {'C': Cs}
x_grid = tf_id_feature.fit_transform(x_train, y_train)
grid = GridSearchCV(LogisticRegression(), param_grid, cv=4)
grid.fit(x_grid, y_train)


C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [42]:
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
print("Best estimator: ", grid.best_estimator_)

Best cross-validation score: 0.90
Best parameters:  {'C': 10}
Best estimator:  LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)


In [44]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(logistic_pipeline, x_set, target_set, cv=4)

C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [45]:
print("Accuracy: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print(scores)

Accuracy: 0.882 (+/- 0.01)
[0.88368 0.88336 0.876   0.88448]


In [63]:
test_path = ".\\data\\test\\"
test_set = []

numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

dir_list = sorted(os.listdir(test_path), key=numericalSort)

for file in dir_list:
    file_path = os.path.join(test_path, file)
    fh = open(file_path, 'r', encoding="utf8")
    entry = {'raw_txt': fh.read()}
    test_set.append(entry)
    fh.close()

test_set = pd.DataFrame(test_set)
print(test_set.head())
print(test_set.shape)


                                             raw_txt
0  Thinking that it could only get better was the...
1  For most people, RoboCop 3 is the film that re...
2  I'm pretty sure Poe would have considered this...
3  This is one of those made-for-TV B movies that...
4  Wallace & Gromit have been around for some tim...
(25000, 1)


In [65]:

test_set = processing(test_set)

test_predictions = pipeline.predict(test_set)

prediction_df = pd.DataFrame(test_predictions)

prediction_df.columns = ['Category']
print(test_set.head())
print(prediction_df.head())

C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


                                             raw_txt  \
0  Thinking that it could only get better was the...   
1  For most people, RoboCop 3 is the film that re...   
2  I'm pretty sure Poe would have considered this...   
3  This is one of those made-for-TV B movies that...   
4  Wallace & Gromit have been around for some tim...   

                                           processed  \
0  thinking that it could only get better was the...   
1  for most people robocop 3 is the film that rea...   
2  im pretty sure poe would have considered this ...   
3  this is one of those madefortv b movies that i...   
4  wallace  gromit have been around for some time...   

                                   text_not_stopword  length  words  \
0  thinking could get better worst assumption eve...     541    106   
1  people robocop 3 film really big disgrace robo...    5125    998   
2  im pretty sure poe would considered travesty f...     336     58   
3  one madefortv b movies awful kind endea

In [66]:
export_csv = prediction_df.to_csv (r'.\test_results.csv', header=True)
